In [ ]:
!pip install PyPDF2

     |████████████████████████████████| 81kB 3.7MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp37-none-any.whl size=61102 sha256=e7eab603af8015ca8ae902f2fa2d9b917e2d558fc378dc6c15a70e9eb37b34d4
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [ ]:
!pip install textract


     |████████████████████████████████| 102kB 4.8MB/s 
     |████████████████████████████████| 8.9MB 9.0MB/s 
     |████████████████████████████████| 112kB 36.3MB/s 
     |████████████████████████████████| 112kB 42.8MB/s 
     |████████████████████████████████| 32.8MB 113kB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 5.6MB 33.7MB/s 
     |████████████████████████████████| 153kB 42.9MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 112kB 47.0MB/s 
     |████████████████████████████████| 1.9MB 29.0MB/s 
  Created wheel for python-pptx: filename=python_pptx-0.6.18-cp37-none-any.whl size=275504 sha256=f0d36368d7fcb2cc7ba5b792a276b0ec6208b5763510493b60b22c70548d48f8
  Stored in directory: /root/.cache/pip/wheels/1f/1f/2c/29acca422b420a0b5210bd2cd7e9669804520d602d2462f20b
  Created wheel for docx2txt: filename=docx2txt-0.8-cp37-none-any.whl size=3981 sha256=bc32a2b4ed99cb64e6a11681f3567283f

In [ ]:
!pip install regex

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls "/content/gdrive/My Drive/Code for San Jose/AlisaViejo_Granicus_01-20-2021.pdf"

'/content/gdrive/My Drive/Code for San Jose/AlisaViejo_Granicus_01-20-2021.pdf'


In [ ]:
import pandas as pd
import numpy as np
import PyPDF2
import textract
import re

In [ ]:
filename="/content/gdrive/My Drive/Code for San Jose/AlisaViejo_Granicus_01-20-2021.pdf"

In [ ]:
pdfFileObj = open(filename,'rb')               #open allows you to read the file
pdfReader = PyPDF2.PdfFileReader(pdfFileObj)   #The pdfReader variable is a readable object that will be parsed
num_pages = pdfReader.numPages                 #discerning the number of pages will allow us to parse through all the pages


count = 0
text = ""
                                                            
while count < num_pages:                       #The while loop will read each page
    pageObj = pdfReader.getPage(count)
    count +=1
    text += pageObj.extractText()
    
#Below if statement exists to check if the above library returned #words. It's done because PyPDF2 cannot read scanned files.

if text != "":
    text = text
    
#If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text

else:
    text = textract.process('http://bit.ly/epo_keyword_extraction_document', method='tesseract', language='eng')

    # Now we have a text variable which contains all the text derived from our PDF file.

In [ ]:
text = text.encode('ascii','ignore').lower() #Lowercasing each word

In [ ]:
text

b" 1 agenda\n aliso viejo\n city council\n  wednesday, january 20, 2021\n 6:00 p.m. closed session\n 7:00 pm regular meeting\n  city hall \n council chambers\n 12 journey \n aliso viejo, ca\n   mayor tiffany ackley\n mayor pro tem ross chun\n councilmember david c. \nharrington\n councilmember richard w. hurt\n councilmember mike munzing\n   special notice regarding covid\n-19    on march 4, 2020, governor newsom proclaimed a state of emergency in california as a \nresult of the threat of covid\n-19. on march 12, 2020, \ngovernor newsom issued executive \norder n\n-25-20, which allows council members to attend city council meetings telephonically. \nplease be advised that some, or all, of the aliso viejo city council members may attend this \nmeeting telephonically.\n    on \nmarch 17, 2020, governor newsom issued executive order n\n-29-20, which allows for the \npublic to participate in any meeting of the city council by electronic means. given the health \nrisks associated with covid

In [ ]:
text = text.decode('ISO-8859-1')

In [ ]:
keywords = re.findall(r'[a-zA-Z]\w+',text)
len(keywords)   

1375

In [ ]:
df = pd.DataFrame(list(set(keywords)),columns=['keywords'])  #Dataframe with unique keywords to avoid repetition in rows


In [ ]:
df.head(10)

,keywords
0,communications
1,pa18
2,broadcast
3,checks
4,with
5,convene
6,except
7,written
8,black
9,scott


In [ ]:
def weightage(word,text,number_of_documents=1):
    word_list = re.findall(word,text)
    number_of_times_word_appeared =len(word_list)
    tf = number_of_times_word_appeared/float(len(text))
    idf = np.log((number_of_documents)/float(number_of_times_word_appeared))
    tf_idf = tf*idf
    return number_of_times_word_appeared,tf,idf ,tf_idf

In [ ]:
df['number_of_times_word_appeared'] = df['keywords'].apply(lambda x: weightage(x,text)[0])
df['tf'] = df['keywords'].apply(lambda x: weightage(x,text)[1])
df['idf'] = df['keywords'].apply(lambda x: weightage(x,text)[2])
df['tf_idf'] = df['keywords'].apply(lambda x: weightage(x,text)[3])

In [ ]:

df = df.sort_values('tf_idf',ascending=True)
df.to_csv('Keywords.csv')
df.head(25)

,keywords,number_of_times_word_appeared,tf,idf,tf_idf
377,on,145,0.013664,-4.976734,-0.068001
362,in,126,0.011873,-4.836282,-0.057423
160,co,118,0.011119,-4.770685,-0.053048
378,an,109,0.010271,-4.691348,-0.048187
13,it,104,0.009800,-4.644391,-0.045516
145,at,102,0.009612,-4.624973,-0.044454
264,or,95,0.008952,-4.553877,-0.040767
374,the,88,0.008292,-4.477337,-0.037128
357,is,62,0.005842,-4.127134,-0.024113
313,of,58,0.005466,-4.060443,-0.022192


In [ ]:
from gensim.summarization import keywords
import warnings
warnings.filterwarnings("ignore")

In [ ]:
values = keywords(text=text,split='\n',scores=True)

In [ ]:
data = pd.DataFrame(values,columns=['keyword','score'])
data = data.sort_values('score',ascending=False)
data.head(10)

,keyword,score
0,citys,0.332018
2,meetings,0.264038
1,meeting,0.264038
3,attachment,0.166304
4,special,0.157443
5,agenda aliso viejo city council,0.152482
6,public,0.151027
7,action,0.149918
8,sections,0.134969
9,comments,0.133481


In [ ]:
!pip install rake_nltk

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7829 sha256=6c6f16b3fe30b770b9c90f2daff6474e0f2a8cd1ba8fef64fa37b4ab852e7c69
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [ ]:
from rake_nltk import Rake

In [ ]:
r = Rake()
r.extract_keywords_from_text(text)

In [ ]:
phrases = r.get_ranked_phrases_with_scores()

In [ ]:
table = pd.DataFrame(phrases,columns=['score','Phrase'])
table = table.sort_values('score',ascending=False)
table.head(10)

,score,Phrase
0,104.000000,ca mayor tiffany ackley mayor pro tem ross chu...
1,87.916667,smith mitzi ortiz city manager city attorney c...
2,74.676389,00 pm regular meeting city hall council chambe...
3,52.375000,hurt councilmember mike munzing special notice...
4,49.727273,95 enterprise ente rprise quarter project reco...
5,41.224026,aliso viejo police services analysis attachment 4
6,40.858225,electronic funds transfers issued decem ber 17
7,40.600000,purchase offer public employee performance eva...
8,40.354167,voted upon immediately following city council ...
9,39.227273,95 enterprise fitness center project recommend...
